Импортируем нужные для работы библиотеки

In [1]:
! pip install spacy
! python -m spacy download ru_core_news_sm

                                              0.0/15.3 MB ? eta -:--:--
                                              0.0/15.3 MB ? eta -:--:--
                                             0.1/15.3 MB 825.8 kB/s eta 0:00:19
                                             0.1/15.3 MB 939.4 kB/s eta 0:00:17
                                              0.2/15.3 MB 1.0 MB/s eta 0:00:15
                                              0.2/15.3 MB 1.3 MB/s eta 0:00:13
                                              0.3/15.3 MB 1.4 MB/s eta 0:00:11
     -                                        0.4/15.3 MB 1.5 MB/s eta 0:00:10
     -                                        0.5/15.3 MB 1.6 MB/s eta 0:00:10
     -                                        0.6/15.3 MB 1.7 MB/s eta 0:00:09
     -                                        0.6/15.3 MB 1.6 MB/s eta 0:00:10
     -                                        0.7/15.3 MB 1.7 MB/s eta 0:00:09
     --                                       0.8/15.3 MB 1.7 MB

In [2]:
import pandas as pd
import unicodedata
import re
import spacy
import json
import numpy as np
from nltk.corpus import stopwords
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import TfidfVectorizer
import os
import json
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt
import gensim
import gensim.corpora as corpora
from gensim.models import CoherenceModel

# Предобработка данных

Загрузим данные

In [3]:
data = pd.read_csv('csv\\full_df.csv', sep=';')
data.head(3)

,Unnamed: 0,status,url,text,likes,reposts,comments
0,0,Middle Software Engineer - Yandex,https://www.linkedin.com/in/michilegorov,Всем привет!Выпустил свою первую статью на хаб...,17.0,0.0,0.0
1,1,Middle Software Engineer - Yandex,https://www.linkedin.com/in/michilegorov,"Если вам интересно позалипать в слова, я запус...",6.0,0.0,2.0
2,2,Middle Software Engineer - Yandex,https://www.linkedin.com/in/michilegorov,"Если вам интересно позалипать в слова, я запус...",17.0,6.0,0.0


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11238 entries, 0 to 11237
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  11238 non-null  int64  
 1   status      11238 non-null  object 
 2   url         11238 non-null  object 
 3   text        11238 non-null  object 
 4   likes       11081 non-null  object 
 5   reposts     10908 non-null  float64
 6   comments    10902 non-null  float64
dtypes: float64(2), int64(1), object(4)
memory usage: 614.7+ KB


In [5]:
data = data.drop(columns='Unnamed: 0', axis=1)

Отфильтруем данные по количеству символов в постах. Оставим посты только длиннее 100 символов.

In [6]:
data['text_len'] = data.loc[:, 'text'].apply(lambda x: len(x))
data_filtered_by_text_len = data.query('text_len > 100')
print(f'Количество постов с 100+ символами: {data_filtered_by_text_len.shape[0]}')
data_filtered_by_text_len.head(3)

Количество постов с 100+ символами: 8556


,status,url,text,likes,reposts,comments,text_len
0,Middle Software Engineer - Yandex,https://www.linkedin.com/in/michilegorov,Всем привет!Выпустил свою первую статью на хаб...,17.0,0.0,0.0,295
1,Middle Software Engineer - Yandex,https://www.linkedin.com/in/michilegorov,"Если вам интересно позалипать в слова, я запус...",6.0,0.0,2.0,117
2,Middle Software Engineer - Yandex,https://www.linkedin.com/in/michilegorov,"Если вам интересно позалипать в слова, я запус...",17.0,6.0,0.0,117


Для обработки нам нужны только посты на кириллице. Отфильтруем посты содержащие кириллицу.

In [7]:
data_cyrillic = data_filtered_by_text_len[data_filtered_by_text_len['text'].apply(lambda x: re.match(r'[А-Яа-я]+', x) is not None)]

print(f'Текстов на кириллице: {data_cyrillic.shape[0]}')
data_cyrillic.head(3)

Текстов на кириллице: 1960


,status,url,text,likes,reposts,comments,text_len
0,Middle Software Engineer - Yandex,https://www.linkedin.com/in/michilegorov,Всем привет!Выпустил свою первую статью на хаб...,17.0,0.0,0.0,295
1,Middle Software Engineer - Yandex,https://www.linkedin.com/in/michilegorov,"Если вам интересно позалипать в слова, я запус...",6.0,0.0,2.0,117
2,Middle Software Engineer - Yandex,https://www.linkedin.com/in/michilegorov,"Если вам интересно позалипать в слова, я запус...",17.0,6.0,0.0,117


Удалим дубликаты в столбце с текстами

In [8]:
data_dd = data_cyrillic.drop_duplicates('text')
print(f'Осталось {data_dd.shape[0]} строк')
data_dd.head(3)

Осталось 1862 строк


,status,url,text,likes,reposts,comments,text_len
0,Middle Software Engineer - Yandex,https://www.linkedin.com/in/michilegorov,Всем привет!Выпустил свою первую статью на хаб...,17.0,0.0,0.0,295
1,Middle Software Engineer - Yandex,https://www.linkedin.com/in/michilegorov,"Если вам интересно позалипать в слова, я запус...",6.0,0.0,2.0,117
6,Talent Acquisition Manager | Recruitment Lead ...,https://www.linkedin.com/in/dariaivanova,С трепетом готов представить свой продукт. Реф...,8.0,0.0,0.0,841


# Подготовка текстов

In [9]:
model = spacy.load('ru_core_news_sm', disable=['ner', 'parser'])

Складываем все посты в один список

In [10]:
texts = data_dd["text"].tolist()

Выполняем лемматизацию текстов. Наберитесь терпения - это долгий процесс, который может занять около часа.

In [11]:
if os.path.exists('lemmas.json'):
    with open("lemmas.json") as f:
        res = json.load(f)['lemmas']
else:
    res = []
    for doc in model.pipe(texts, disable=["tagger", "parser"]):
        res.append([token.lemma_ for token in doc])

    with open("lemmas.json", "w") as fid:
        json.dump({"lemmas": res}, fid)

Смотрим на результат - случайным образом сэмплируем несколько примеров

In [12]:
", ".join(res[np.random.randint(len(res))])

'консультировать, ряд, компания, на, предмет, внедрение, data, -, driven, decisions, заметить, интересный, вещь, ,, которая, являться, предпосылка, ", успех, ", от, внедрение, какой, -, либо, бизнес, -, процесса, и, проект, ., весь, мы, знать, ,, что, в, целое, успех, в, компания,  , в, больший, свой, часть, зависеть, от:-, архитектор, ,, который, придумывать, архитектура, бизнес, ,, он, стратегия, ,, говорить, на, какой, рынок, и, какой, бизнес, модель, работать,  , -, от, оперативный, руководитель, ,, то, быть, человек, ,, который, управлять, рука, и, голова, человек, ,, ставить, им, задача, и, контролировать, правильность, управления-, ну, и, очевидно, ,, от, исполнитель, ,, тот, кто, реализовать, весь, этот, продукт, ., так, вот, ,, я, заметить, ,, что, если, оперативный, руководитель, иметь, хороший, опыт, исполнитель, ,, то, быть, свой, же, рука, делать, проект, ,, а, после, что, ,, осознанно, понять, ,, что, он, hard, skills, мало, ,, чтобы, перейти, от, исполнитель, к, оператив

In [13]:
", ".join(res[np.random.randint(len(res))])

'лендинг, пейдж, под, ключ, ., инвестиция, в, коммерческий, и, жилой, недвижимость, ., ipc, group, ., #, landingpage, #, webdesign'

Удалим знаки препинания, цифры и слова, содержащие цифры. Для это соберём регулярное выражение, которое подходит для любого слова, состоящего из букв кирилицы.

In [14]:
word_pattern = re.compile("^[а-я]*$")

def remove_symbols(doc):
    return [token for token in doc if word_pattern.match(token)]

corpus = list(map(remove_symbols, res))

In [15]:
", ".join(corpus[np.random.randint(len(corpus))])

'как, ускорить, медленный, приложение, разбирать, неочевидный, причина, медленный, работа, код, и, способ, он, оптимизация, в, это, видео, забывать, делиться, ваш, рекомендация, в, комментарий, разработка'

# Векторизация текстов

Теперь необходимо перевести тексты новостей и слова, из которых они состоят, в числовое представление, т.е. провести векторизацию. Для этого можно использовать подход "мешок слов" (Bag of Words) или Tf-iDf. Второй вариант нравится мне больше.

Чтобы уменьшить размерность получаемой матрицы, добавим исключение стоп-слов.

In [ ]:
stop_words = stopwords.words('russian')
print(stop_words)

In [ ]:
tfidf = TfidfVectorizer(stop_words=stop_words, min_df=10, max_df=0.9)

Т.е. TfIdfVectorizer принимает на вход список предложений, поэтому необходимо объединить списки слов, составляющих корпус, в предложения.

In [ ]:
docs = [" ".join(tokens) for tokens in corpus]
docs[:3]

In [ ]:
x = tfidf.fit_transform(docs)

In [ ]:
x.shape

Теперь составим словарь {id_токена: токен} - он пригодится нам позднее.

In [ ]:
id2word = {i: token for i, token in enumerate(tfidf.get_feature_names_out())}

# LDA

Теперь можем запустить алгоритм LDA. Зададим количество тем равное 20.

In [ ]:
N_TOPICS = 20

In [ ]:
lda = LatentDirichletAllocation(n_components=N_TOPICS, random_state=765)

In [ ]:
topics = lda.fit_transform(x)

In [ ]:
topics.shape

Получили массив размера 1862x20. Строки здесь - отдельные тексты, а колонки - 20 выделенных тем. В каждой ячейке стоит вероятность того, что данный текст относится к данной теме. Это то самое распределение тем над документами.

## Ключевые слова

Теперь извлечём ключевые слова для каждой из 20 тем. В свойсте components _ лежит матрица соответствия слов темам. Пройдёмся по каждой из 20 выделенных тем и для каждой возьмём ТОП-20 слов.

По очереди пройдёмся по каждой теме (первое измерение массиа) и выберем ТОП-20 слов, наиболее характерных для этой темы. Потом по индексу слова обратимся к словарю и узнаем, что это за слово.

In [ ]:
for i in range(N_TOPICS):
    c = lda.components_[i, :]
    topic_tokens = [id2word.get(token_id) for token_id in np.argsort(c)[-20:]]
    print("Тема",i+1, ":")
    print(", ".join(topic_tokens))
    print("\n ")

По полученным ключевым словам можно составить представление о содержании той или иной темы. Хотя иногда сочетания слов довольно странные )

## Типичные статьи

Теперь для каждой темы возьмём по одному документу, который хорошо соответсвует этой теме. Для этого заглянем в массив topics.

In [ ]:
for i in range(N_TOPICS):
    doc_id = np.argmax(topics[:, i])
    print("Тема ", i+1)
    print(data_dd.iloc[doc_id]["text"])
    print("\n")

# Тест

In [ ]:
def lda(n_topics):
    lda = LatentDirichletAllocation(n_components=n_topics, random_state=765)
    topics = lda.fit_transform(x)
    print(topics.shape)

    for i in range(n_topics):
        c = lda.components_[i, :]
        topic_tokens = [id2word.get(token_id) for token_id in np.argsort(c)[-20:]]
        print("Тема",i+1, ":")
        print(", ".join(topic_tokens))
        print("\n ")

    for i in range(n_topics):
        doc_id = np.argmax(topics[:, i])
        print("Тема ", i+1)
        print(data_dd.iloc[doc_id]["text"])
        print("\n")